**Setup**

In [1]:
import numpy as np
import pandas as pd

import os
import datetime
import pickle
import random
import matplotlib.pyplot as plt

from sentinelhub import SHConfig

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    MimeType,
    SentinelHubRequest,
    bbox_to_dimensions,
    SentinelHubCatalog,
    SentinelHubDownloadClient,
    MosaickingOrder,
)

/home/laurens/anaconda3/envs/AutoRTM_data/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Parameters

overwrite = False
save_path = "/mnt/c/Users/Laurens/Downloads/test/"
load_meta = True # Load previous image_ids, otherwise create new (overwrite if existing)
save_meta = True # Save found image_ids, overwrite if exists

time_interval = ("2018-01-01", "2023-12-31") # Should be January 2018 at earliest, availability is 2017 and need 1 year past image
time_diff_allowance = {
    '1w': (3, 11),
    '1m': (20, 45),
    '3m': (75, 105),
    '6m': (150, 210),
    '12m': (320, 410),
    'mask': (-180, 180),
}
cloud_mask_min_cover_percentage = 5
cloud_mask_max_cover_percentage = 30
locations_plot_size = 0.4

# Locations

location_points = {
    "farmland_ukraine":[30.066, 48.833],
    "desert_california": [-113.12368580737437, 27.441480787333745],
    "northpole_canada": [-79.35669362734849, 66.84722100064704],
    "desert_southafrica": [25.52310069192609, -33.056843724060336],
    "forest_zimbabwe": [30.58798983978874, -17.73756749163565],
    "plains_tibet": [91.13203134288163, 31.075922872630954],
    "hills_spain": [-1.190488095366975, 39.952996851734966],
    "farmland_bosnia": [18.424610094643334, 45.292629729578664],
    "suburb_china": [119.80484720313598, 31.709066215967937],
    "plains_kazakhstan": [80.0734979625584, 48.89985347669314],
    "forest_china": [114.68157913680628, 31.796948689077283],
    "rural_france": [3.3923301662956256, 45.26738815730779],
    "river_serbia": [20.843948268287463, 44.65642844630237],
    "shrubland_russia": [133.17428356135662, 43.076673006524445],
    "farmland_france": [5.3004647240852485, 47.32525323856271],
    "forest_angola": [20.3633310505651, -7.716179536707344],
    "farmland_pakistan": [72.07380655741467, 33.52465740882829],
    "mixed_urban_farmland_sudan": [32.51612072356935, 15.6300542829119],
    "farmland_bangladesh": [88.48588881575397, 25.66982981619917],
    "farmland_china": [120.15373376483726, 36.61395705115041],
    "mixed_forest_farmland_bangladesh": [89.18531426060684, 22.717274989260517],
    "urban_incheon": [126.64570197185921, 37.55259819528162],
    "farmland_thailand": [102.34451882612518, 17.6213733436621],
    "forest_myanmar": [96.97689093340311, 20.567045711115334],
    "shrubland_thailand": [102.11647571015789, 17.666403526514493],
}

locations = {}
for k,v in location_points.items():
    coords = {
        "min_x":v[0] - locations_plot_size/2,
        "max_x":v[0] + locations_plot_size/2,
        "min_y":v[1] - locations_plot_size/2,
        "max_y":v[1] + locations_plot_size/2,
    }
    locations[k] = coords



In [3]:
# SH setup

config = SHConfig()
catalog = SentinelHubCatalog(config=config)

# Evalscript

evalscript = """
    //VERSION=3

    function setup() {
        return {
            input: [
                {bands: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12", 
                            "CLP", "CLM", "dataMask"]},
            ],
            output: {
                bands: 13,
                sampleType: "FLOAT32"
            }
        };
    }

    function evaluatePixel(sample) {
        if(sample.dataMask == 1) {
            if(sample.CLM != 255) {
                return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B08, 
                        sample.B8A, sample.B09, sample.B11, sample.B12, sample.CLP/255.0];
            }
            else {
                return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B08, 
                        sample.B8A, sample.B09, sample.B11, sample.B12,-999.0];
            }
        }
        else {
            return [-999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, 
                    -999.0, -999.0, -999.0, -999.0, -999.0];
        }
    }
"""


evalscript_s1 = """
    //VERSION=3

    function setup() {
        return {
            input: [
                {bands: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12", 
                            "CLP", "CLM", "dataMask"]},
            ],
            output: {
                bands: 14,
                sampleType: "FLOAT32"
            }
        };
    }

    function evaluatePixel(sample) {
        if(sample.dataMask == 1) {
            if(sample.CLM != 255) {
                return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B08, 
                        sample.B8A, sample.B09, sample.B10, sample.B11, sample.B12, sample.CLP/255.0];
            }
            else {
                return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B08, 
                        sample.B8A, sample.B09, sample.B10, sample.B11, sample.B12,-999.0];
            }
        }
        else {
            return [-999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, -999.0, 
                    -999.0, -999.0, -999.0, -999.0, -999.0, -999.0];
        }
    }
"""

evalscript_preview = """
    //VERSION=3

    function setup() {
        return {
            input: [
                {bands: ["B04", "dataMask"]},
            ],
            output: {
                bands: 1,
                sampleType: "FLOAT32"
            }
        };
    }

    function evaluatePixel(sample) {
        if(sample.dataMask == 1) {
            return [sample.B04];
        }
        else {
            return [-999.0];
        }
    }
"""



def missing_data_check(image):
    # Simple for now, if CLM==255 --> CLP got saved as -999.0, if dataMask==0 --> bands saved as -999.0
    # So if any element has value -999.0, do not use image
    if((image.flatten() == -999.0).any()):
        return(False)
    else:
        return(True)


def check_valid_image(result_list, base_datetime, time_diff_allowance, start_index=0):

    start_index = 0
    for i in range(start_index, len(result_list)):
        candidate_w1 = result_list[i]
        dt_w1 = datetime.datetime.strptime(candidate_w1['properties']['datetime'], "%Y-%m-%dT%H:%M:%SZ")
        time_delta = base_datetime - dt_w1 

        if(time_delta.days < time_diff_allowance[0]):
            # Not close enough yet in time, go to next candidate
            pass
        elif(time_delta.days > time_diff_allowance[1]):
            # Stop looking once we get to a time difference that is too far away
            return(False, None, None)
        else:
            # No need for cloud cover check since we use the same results list
            return(True, candidate_w1['id'], i)

    return(False, None, None)
    

def normalise_and_visualise(img,title="",rgb=[3,2,1],percentile_clip=True,save_fig=False,**kwargs):
    
    new_img = np.zeros((img.shape[0],img.shape[1],3))
    new_img[:,:,0] = img[:,:,rgb[0]]
    new_img[:,:,1] = img[:,:,rgb[1]]
    new_img[:,:,2] = img[:,:,rgb[2]]
    
    if(percentile_clip):
        min_val = np.nanpercentile(new_img,1)
        max_val = np.nanpercentile(new_img,99)

        new_img = np.clip((new_img-min_val)/(max_val-min_val),0,1)
    
    plt.imshow(new_img,interpolation="nearest")
    plt.title(title)
    plt.axis('off')
    if(save_fig):
        plt.savefig(kwargs['path'],bbox_inches='tight')
    plt.show()


def normalise_and_visualise_1d(img,title="",percentile_clip=True,save_fig=False,path='saved_fig.pdf',**kwargs):
    
    new_img = np.zeros((img.shape[0],img.shape[1]))
    new_img[:,:] = img[:,:]
    
    if(percentile_clip):
        min_val = np.nanpercentile(new_img,1)
        max_val = np.nanpercentile(new_img,99)

        new_img = np.clip((new_img-min_val)/(max_val-min_val),0,1)
    
    plt.imshow(new_img,interpolation="nearest",**kwargs)
    plt.title(title)
    plt.axis('off')
    if(save_fig):
        plt.savefig(path,bbox_inches='tight')
    plt.show()



def check_if_subset(bbox_original, geometry):
    bbox_coords = list(bbox_original)
    # [lon_min, lat_min, lon_max, lat_max]
    b_lon_min = bbox_coords[0]
    b_lon_max = bbox_coords[2]
    b_lat_min = bbox_coords[1]
    b_lat_max = bbox_coords[3]

    result_coords = np.array(geometry['coordinates'][0][0])
    # [lon, lat]
    r_lon_min = np.min(result_coords[:, 0])
    r_lon_max = np.max(result_coords[:, 0])
    r_lat_min = np.min(result_coords[:, 1])
    r_lat_max = np.max(result_coords[:, 1])

    if(b_lon_min > r_lon_min and b_lon_max < r_lon_max):
        if(b_lat_min > r_lat_min and b_lat_max < r_lat_max):
            return(True)

    return(False)

In [3]:
# Separated from other cell to keep previous runs of cell

to_download = {}

if(load_meta):
    with open("image_ids_unsorted.pkl", 'rb') as fp:
        to_download = pickle.load(fp)

print("To download:")
for k,v in to_download.items():
    print({k:v})

NameError: name 'load_meta' is not defined

**Extending existing dict with extra data for NN baseline**

In [4]:
# Extra cell added after previous data downloads, to add L1C product and SAR data for NN baseline

with open("image_ids_unsorted.pkl", 'rb') as fp:
    to_download = pickle.load(fp)

with open("coord_dict.pkl", 'rb') as fp:
    coord_dict = pickle.load(fp)

to_download2 = {}
for scene, sd in to_download.items():
    #print(scene)
    # Get rough bbox based on centre coordinates
    location_point = coord_dict[scene]
    location_coords = {
        "min_x":location_point[1] - 0.1/2,
        "max_x":location_point[1] + 0.1/2,
        "min_y":location_point[0] - 0.1/2,
        "max_y":location_point[0] + 0.1/2,
    }
    bbox = BBox(bbox=location_coords, crs=CRS.WGS84)

    # Iterate over products for this scene, only do more than copying for the mask image
    sd_new = {}
    for img, name in sd.items():
        if(not(img == "sar" or img == "l1c")):
            sd_new[img] = name # copy all existing names
        if(img == "target"):
            # Set existing product's properties
            timestamp_str = name[11:19]
            base_datetime = datetime.datetime.strptime(timestamp_str, "%Y%m%d")

            # Search for corresponding L1C product
            time_interval = (base_datetime - datetime.timedelta(days=2), base_datetime + datetime.timedelta(days=2))

            search_iterator = catalog.search(
                DataCollection.SENTINEL2_L1C,
                bbox=bbox,
                time=time_interval,
                fields={"include": ["id"], "exclude": []},
            )
            results_l1c = list(search_iterator)
            #print(len(results_l1c))

            # Search for corresponding SAR product
            time_interval = (base_datetime - datetime.timedelta(days=10), base_datetime + datetime.timedelta(days=10))

            search_iterator = catalog.search(
                DataCollection.SENTINEL1_IW,
                bbox=bbox,
                time=time_interval,
                fields={"include": ["id"], "exclude": []},
            )
            results_sar = list(search_iterator)
            #print(len(results_sar))

            # Add new products to scene dict new (sd_new)
            if(len(results_l1c) > 0):
                sd_new["l1c"] = results_l1c[0]['id']
            else:
                print("Warning: no L1C data found for scene ", scene)
                sd_new["l1c"] = ""
            if(len(results_sar) > 0):
                sd_new["sar"] = results_sar[0]['id']
            else:
                print("Warning: no SAR data found for scene ", scene)
                sd_new["sar"] = ""

    # Add new scene dict to to_download2
    to_download2[scene] = sd_new

print("To download updated:")
for k,v in to_download2.items():
    print({k:v})

To download updated:
{'asia_cropland_china': {'target': 'S2B_MSIL2A_20211228T031129_N0301_R075_T50SKB_20211228T054139', 'l1c': 'S2B_MSIL1C_20211228T031129_N0301_R075_T50SKB_20211228T050126', 'sar': 'S1A_IW_GRDH_1SDV_20220104T102821_20220104T102846_041310_04E925_2896', '1w': 'S2B_MSIL2A_20211218T031129_N0301_R075_T50SKA_20211218T054114', '1m': 'S2B_MSIL2A_20211205T030109_N0301_R032_T50SKA_20211205T061129', '3m': 'S2A_MSIL2A_20211004T030551_N0301_R075_T50SKA_20211004T051723', '6m': 'S2A_MSIL2A_20210606T030541_N0300_R075_T50SKA_20210606T051217', '12m': 'S2A_MSIL2A_20210203T025931_N0214_R032_T50SKA_20210203T055016', 'mask': 'S2B_MSIL2A_20220226T030659_N0400_R075_T50SKB_20220226T055944'}}
{'america_cropland_iowa': {'target': 'S2A_MSIL2A_20220623T170901_N0400_R112_T15TVJ_20220623T231619', 'l1c': 'S2A_MSIL1C_20220623T170901_N0400_R112_T15TVJ_20220623T210031', 'sar': '', '1w': 'S2A_MSIL2A_20220620T165901_N0400_R069_T15TVJ_20220621T002016', '1m': 'S2A_MSIL2A_20220603T170901_N0400_R112_T15TVJ_20

In [6]:
# WARNING: use with caution (overwrites img_ids to add extra data to it)

with open('image_ids_unsorted.pkl', 'wb') as fp:
    pickle.dump(to_download2, fp)

**Defining extents for certain land types in different areas to search in**

In [9]:
# Define multiple search extents per land use class

extents_urban_built_up = {
    "europe_urban_paris":[48.625,2.203,49.001,2.633],
    "asia_urban_seoul":[37.483,126.726,37.593,127.092],
    "america_urban_newyork":[40.476,-74.548,41.019,-72.974],
    "africa_urban_lagos":[6.414,3.155,6.770,3.618],
    "europe_urban_rome":[41.831,12.405,41.948,12.601],
    "europe_urban_london":[51.310,-0.410,51.625,0.128],
    "europe_urban_rotterdam":[51.853,4.253,51.980,4.640],
    #"europe_urban_madrid":[40.305,-3.825,40.569,-3.587],
    "asia_urban_tokyo":[35.301,139.297,35.972,140.079],
    "asia_urban_shanghai":[30.982,121.175,31.448,121.7244],
    "asia_urban_taipei":[24.986,121.407,25.087,121.6050],
    #"asia_urban_beijing":[39.695,116.077,40.176,116.705],
    "america_urban_boston":[42.261,-71.250,42.537,-70.943],
    #"america_urban_atlanta":[33.707,-84.609,34.085,-83.916],
    #"america_urban_fortworth":[32.642,-97.437,33.118,-96.528],
    "europe_urban_moscow":[55.612,37.345,55.950,38.002],
    "asia_urban_jakarta":[-6.382,106.541,-6.096,107.079],
    "america_urban_mexicocity":[19.265,-99.291,19.695,-98.954],
    "asia_urban_tehran":[35.633,51.253,35.810,51.550],
    #"asia_urban_cairo":[29.948,31.093,30.196,31.480],
    "australia_urban_sydney":[-33.984,150.834,-33.712,151.313],
    "australia_urban_melbourne":[-38.004,114.736,-37.639,145.379],
    "africa_urban_ouagadougou":[12.302,-1.628,12.438,-1.397],
    #"asia_urban_riyadh":[24.556,46.625,24.830,46.864],
    #"asia_urban_newdelhi":[28.411,76.980,28.753,77.515],
    "asia_urban_bangkok":[13.567,100.355,14.031,100.773],
    "asia_urban_hongkong":[22.467,112.934,23.188,114.096],
    "america_urban_phoenix":[33.422,-112.406,33.708,-111.872],
    "america_urban_saltlakecity":[40.482,-112.035,40.774,-111.808],
    "america_urban_houston":[29.551,-95.718,30.070,-95.154],
    "africa_urban_khartoum":[15.448,32.371,15.742,32.640],
    "asia_urban_baghdad":[33.220,44.275,33.425,44.547],
    "asia_urban_hochiminhcity":[10.700,106.585,11.003,106.841],
    "asia_urban_manila":[14.407,120.922,14.735,121.159],
    "asia_urban_kaohsiung":[22.516,120.201,23.043,120.348],
    "europe_urban_naples":[40.709,14.056,40.962,14.598],
}

extents_cropland = {
    "europe_cropland_ukraine":[46.337,27.572,49.542,37.526],
    #"asia_cropland_china":[32.188,111.969,39.848,122.135],
    #"america_cropland_iowa":[41.717,-96.079,44.185,-92.827],
    "africa_cropland_nile":[30.730,30.723,31.375,31.818],
    "europe_cropland_italynorth":[44.463,7.544,45.319,12.532],
    "europe_cropland_hungary":[45.355,18.424,47.599,22.416],
    "europe_cropland_caucasus":[44.003,37.863,46.747,44.982],
    "europe_cropland_russia":[50.651,35.856,52.792,52.145],
    #"asia_cropland_india":[14.371,73.890,28.159,81.929],
    "asia_cropland_thailand":[14.509,99.532,17.770,105.696],
    "asia_cropland_cambodia":[9.825,104.200,12.549,106.536],

}

extents_forest = {
    "europe_forest_romania":[46.687,24.979,47.872,26.089],
    "asia_forest_borneo":[-1.894,110.918,4.511,117.092],
    "america_forest_amazon":[-7.382,-77.311,0.548,-55.887],
    "africa_forest_congobasin":[-2.670,18.541,5.219,28.231],
    "europe_forest_ukraine":[51.412,27.920,51.650,30.298],
    "europe_forest_russia":[59.583,42.825,64.956,58.436],
    "asia_forest_russia":[55.414,97.029,63.206,107.706],
    "asia_forest_laos":[20.280,100.602,23.109,102.799],
    "asia_forest_malaysiasouth":[3.494,101.456,5.521,102.641],
    "asia_forest_malaysianorth":[8.459,98.479,15.992,99.338],
    "africa_forest_gabon":[-0.266,10.749,4.127,18.558],
    #"africa_forest_zambia":[-14.049,23.333,-11.293,27.013],
    "america_forest_guatemala":[17.333,-90.319,19.305,-88.759],
    "america_forest_louisiana":[30.248,-94.669,33.914,-91.936],
    #"america_forest_mississippi":[30.576,-90.987,34.415,-85.221],
    "america_forest_westvirginia":[36.616,-83.452,39.936,-78.113],
    #"america_forest_oregon":[41.003,-123.915,44.016,-121.619],
    #"america_forest_quebec":[43.721,-70.431,47.982,-64.466],
    "america_forest_ontario":[46.896,-86.698,48.952,-72.857],
    "europe_forest_swedensouth":[56.565,13.074,58.057,16.732],
    "europe_forest_swedenmiddle":[59.561,12.162,61.579,17.075],
    "america_forest_missouri":[35.470,-93.083,38.342,-90.381],
    "america_forest_kentucky":[36.645,-83.558,39.448,-78.160],
    "america_forest_pennsylvania":[41.143,-79.801,42.548,-72.792],
    "america_forest_newyork":[43.240,-75.333,44.874,-69.987],
    "america_forest_newbrunswick":[46.073,-67.299,47.980,-64.691],
    "america_forest_canada":[45.982,-80.394,47.833,-69.902],
    "america_forest_wisconsin":[45.223,-91.403,46.895,-86.971],
    "america_forest_minnesota":[46.256,-94.662,48.466,-91.531],    
    "asia_forest_hainan":[18.474,108.939,19.503,110.614],
    "asia_forest_vietnam":[15.031,106.108,17.545,108.847],
    "asia_forest_vietnamnorth":[18.705,101.049,23.117,105.802],
    "asia_forest_guizhou":[24.590,107.369,29.828,111.229],
    "asia_forest_shaanxi":[32.123,104.971,34.007,112.192],
    "asia_forest_fujian":[24.817,116.949,28.062,120.289],
    "asia_forest_buryatia":[48.956,107.380,51.815,113.687],
    "asia_forest_manchuria":[50.462,118.293,56.909,125.021],
    "asia_forest_sakha":[58.587,117.532,63.131,137.241],
    "europe_forest_genoa":[43.899,8.099,44.779,11.142],
    "europe_forest_spain":[37.203,-4.465,38.634,-2.718],
    "europe_forest_spainnorth":[42.810,-7.214,43.220,-0.831],
    "asia_forest_northkorea":[40.203,124.194,42.302,129.973],
    "asia_forest_primorskykrai":[43.057,135.090,50.895,140.563],
    "asia_forest_hokkaido":[41.427,139.678,45.447,145.556],
    "africa_forest_angola":[-15.334,17.501,-10.288,20.786],
    "africa_forest_mozambique":[-17.627,35.475,-8.707,40.649],
    "africa_forest_eastafrica":[4.577,-12.933,10.177,-3.287],   
}

extents_herbaceous = {
    "europe_herbaceous_ireland":[52.320,-9.434,55.118,-6.306],
    #"asia_herbaceous_kazakhstan":[46.284,66.233,49.002,81.760],
    "america_herbaceous_bolivia":[-19.394,-66.636,-17.738,-64.856],
    #"africa_herbaceous_southafrica":[-33.700,23.555,-29.424,28.345],
    "europe_herbaceous_switzerland":[46.187,7.335,47.331,11.158],
    "australia_herbaceous_north":[-22.343,119.594,-18.754,128.346],
    #"america_herbaceous_peru":[-15.185,-74.454,-13.461,-68.906],
    "america_herbaceous_colombia":[-4.215,-71.803,8.451,-67.365],
    "america_herbaceous_venezuela":[7.450,-70.920,9.557,-62.922],
    "america_herbaceous_newmexico":[33.679,-105.209,39.073,-103.184],
    "asia_herbaceous_shanxi":[36.409,106.714,40.134,111.420],
    "asia_herbaceous_mongoliaeast":[44.166,113.119,50.804,119.487],
    "asia_herbaceous_mongolia":[46.483,96.218,48.147,111.581],
}

extents_shrubs = {
    "europe_shrubs_norway":[59.753,6.577,61.360,8.628],
    #"asia_shrubs_pakistan":[27.868,70.928,28.777,72.334],
    #"america_shrubs_mexico":[25.938,-104.714,30.265,-101.323],
    "africa_shrubs_kenia":[-0.304,39.668,4.087,45.194],
    "australia_shrubs_south":[-31.444,132.825,-30.294,134.945],
    #"australia_shrubs_west":[-29.844,117.960,-28.221,121.893],
    "america_shrubs_peru":[-6.388,-81.119,-4.496,-79.933],
    "america_shrubs_colombia":[11.533,-72.827,12.414,-71.067],
}

land_cover = "forest"


**Search for suitable sets of products, add IDs of suitable sets to dict for later download**

In [10]:
# Exploratory cell, used to find some candidate locations
# Can be used for inspiration, for example with a bbox over a continent, when using manual_extent.
# Can also be used to dataset_specification.csv, where more specific bboxes are pre-defined (e.g.,
# over the Amazon rainforest), and the code can be used to search for suitable image combinations.

from IPython.display import clear_output # For a nicer interface in this cell

num_success = 0
max_success = 1
max_iter = 2000

manual_extent = False

if(manual_extent):
    min_x_coord = 0.0
    max_x_coord = 44.446
    
    min_y_coord = 47.432
    max_y_coord = 53.139

else:
    max_success = 1




location_points = {}
locations = {}

found_any = False

for _ in range(0, max_iter):

    if(num_success >= max_success):
        print("Reached maximum number of suitable scenes.")
        break

    if(not(manual_extent)):
        k = None
        v = None
        if(land_cover == "urban_built_up"):
            k,v = random.choice(list(extents_urban_built_up.items()))
        elif(land_cover == "cropland"):
            k,v = random.choice(list(extents_cropland.items()))
        elif(land_cover == "forest"):
            k,v = random.choice(list(extents_forest.items()))
        elif(land_cover == "herbaceous"):
            k,v = random.choice(list(extents_herbaceous.items()))
        elif(land_cover == "shrubs"):
            k,v = random.choice(list(extents_shrubs.items()))

        scene_name = k
        min_x_coord = v[1]
        max_x_coord = v[3]
        min_y_coord = v[0]
        max_y_coord = v[2]

        width_x_coords = max_x_coord - min_x_coord
        height_y_coords = max_y_coord - min_y_coord

    location_point = [np.random.rand()*width_x_coords + min_x_coord, np.random.rand()*height_y_coords + min_y_coord]
    location_coords = {
        "min_x":location_point[0] - locations_plot_size/2,
        "max_x":location_point[0] + locations_plot_size/2,
        "min_y":location_point[1] - locations_plot_size/2,
        "max_y":location_point[1] + locations_plot_size/2,
    }


    # Search catalog for suitable combinations of images to download (actually download later)


    # Iterate over locations, search for appropriate data

    catalog = SentinelHubCatalog(config=config)
    catalog.get_collection(DataCollection.SENTINEL2_L2A)


    # Setup
    resolution = 20
    bbox = BBox(bbox=location_coords, crs=CRS.WGS84)
    size = bbox_to_dimensions(bbox, resolution=resolution)

    # Search for suitable combinations

    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        bbox=bbox,
        time=time_interval,
        filter="eo:cloud_cover < 0.1",
        fields={"include": ["id", "geometry", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
    )

    results = list(search_iterator)

    # Check results for suitable images in past time steps

    scene_ids = {}

    for candidate in results:

        scene_ids = {}
        success = True # If no suitable combination is found at any point, set to False

        # Check for coverage polygon
        result_geometry = candidate['geometry']
        subset = check_if_subset(bbox, result_geometry)
        if(not(subset)):
            success = False

        base_image = candidate
        base_id = base_image['id']
        scene_ids['target'] = base_id
        base_datetime = datetime.datetime.strptime(base_image['properties']['datetime'], "%Y-%m-%dT%H:%M:%SZ")

        # Inefficient nested loop, but should be fine since we only do this a few times

        end_index = -1
        
        if(success):
            # 1 week
            success, img_id, end_index = check_valid_image(results, base_datetime, time_diff_allowance['1w'], start_index=0)
            if(success):
                scene_ids['1w'] = img_id

        if(success):
            # 1 month
            success, img_id, end_index = check_valid_image(results, base_datetime, time_diff_allowance['1m'], start_index=end_index)
            if(success):
                scene_ids['1m'] = img_id

        if(success):
            # 3 months
            success, img_id, end_index = check_valid_image(results, base_datetime, time_diff_allowance['3m'], start_index=end_index)
            if(success):
                scene_ids['3m'] = img_id

        if(success):
            # 6 months
            success, img_id, end_index = check_valid_image(results, base_datetime, time_diff_allowance['6m'], start_index=end_index)
            if(success):
                scene_ids['6m'] = img_id

        if(success):
            # 12 months
            success, img_id, end_index = check_valid_image(results, base_datetime, time_diff_allowance['12m'], start_index=end_index)
            if(success):
                scene_ids['12m'] = img_id

        if(success):
            # mask
            # This needs a new search, because cloud cover must be high instead of low
            mask_time_interval = (base_datetime - datetime.timedelta(days=time_diff_allowance['mask'][0]), base_datetime + datetime.timedelta(days=time_diff_allowance['mask'][1]))

            search_iterator2 = catalog.search(
                DataCollection.SENTINEL2_L2A,
                bbox=bbox,
                time=mask_time_interval,
                filter="eo:cloud_cover > " + str(cloud_mask_min_cover_percentage),
                fields={"include": ["id", "geometry", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
            )

            results2 = list(search_iterator2)

            # Some automated checks for properties; remove instances that are unsuitable
            fail_indices = []
            i = 0
            for mask_result in results2:
                cc = mask_result['properties']['eo:cloud_cover']
                if(cc < cloud_mask_min_cover_percentage or cc > cloud_mask_max_cover_percentage):
                    fail_indices.append(i)
                else:
                    result_geometry = mask_result['geometry']
                    subset = check_if_subset(bbox, result_geometry) # May be superfluous now
                    if(not(subset)):
                        fail_indices.append(i)
                i += 1
            results2 = [results2[i] for i in range(0, len(results2)) if not(i in fail_indices)]

            # Check if any (remaining) results match
            if(len(results2) == 0):
                success = False


        if(success):
            # Download previews, plot and ask if worth keeping

            print("Downloading previews")

            print("Coordinates: ", location_point)
            flipped = "[" + str(location_point[1]) + ", " + str(location_point[0]) + "]"
            print("Coordinates flipped: ", flipped)

            #preview_approval = input("Show previews for " + flipped + "? (y/n/q, default=y)")
            #if(preview_approval == "n"):
            #    break
            #elif(preview_approval == "q"):
            #    dlfkjsdlkfj

            for feature_type, scene_id in scene_ids.items():     

                timestamp_str = scene_id[11:19]
                timestamp = datetime.datetime.strptime(timestamp_str, "%Y%m%d")
                interval = (timestamp - datetime.timedelta(days=2), timestamp + datetime.timedelta(days=2))

                bbox = BBox(bbox=location_coords, crs=CRS.WGS84)
                size = bbox_to_dimensions(bbox, resolution=180)

                request = SentinelHubRequest(
                    evalscript=evalscript_preview,
                    input_data=[
                        SentinelHubRequest.input_data(
                            data_collection=DataCollection.SENTINEL2_L1C,
                            time_interval=interval,
                            other_args={'previewMode':'PREVIEW'}, # doesn't seem to work...
                        )
                    ],
                    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
                    bbox=bbox,
                    size=size,
                    config=config,
                )
                print("Getting data for ", feature_type)
                data = request.get_data()
                preview_img = data[0]

                # Check whether these are partial images or not
                if(not(missing_data_check(preview_img))):
                    success = False
                    break
                else:
                    normalise_and_visualise_1d(preview_img, title=feature_type, save_fig=False, path=feature_type+".pdf")

            if(success):
                print("\nFound all necessary images for candidate scene.")
                print("Coordinates: ", location_point)
                print("Coordinates flipped: ", "[" + str(location_point[1]) + ", " + str(location_point[0]) + "]")
                user_approval = input("Keep the above scene: " + str(scene_name) + "? (y/n/q, default: y)")
                if(user_approval == "q" or user_approval == "e"):
                    dslkjf # crash
                elif(user_approval == "n"):
                    success = False
                else:
                    if(manual_extent):
                        scene_name = input("Please enter a descriptive name for this scene: ")


            if(success):
                # Find nice cloud mask; keep this outside main loop so we don't have to discard other stuff
                # when looking for a suitable mask
                found_mask = False
                for mask_result in results2:
                    #candidate_mask_timestep_str = mask_result['properties']['datetime']
                    candidate_mask_timestep_str = mask_result['id'][11:19]
                    candidate_mask_timestep = datetime.datetime.strptime(candidate_mask_timestep_str, "%Y%m%d")
                    interval = (candidate_mask_timestep - datetime.timedelta(days=2), candidate_mask_timestep + datetime.timedelta(days=2))

                    # Download mask preview
                    request = SentinelHubRequest(
                        evalscript=evalscript_preview,
                        input_data=[
                            SentinelHubRequest.input_data(
                                data_collection=DataCollection.SENTINEL2_L1C,
                                time_interval=interval,
                                other_args={'previewMode':'PREVIEW'}, # doesn't seem to work...
                            )
                        ],
                        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
                        bbox=bbox,
                        size=size,
                        config=config,
                    )
                    print("Getting data for mask")
                    data = request.get_data()
                    preview_img = data[0]

                    # Check whether these are partial images or not
                    if(not(missing_data_check(preview_img))):
                        success = False
                        break
                    else:
                        normalise_and_visualise_1d(preview_img, title="Candidate mask")

                    user_approval2 = input("Keep this cloud mask? (y/n/q, default=n)")

                    if(user_approval2 == "y"):
                        scene_ids['mask'] = mask_result['id']
                        found_mask = True
                        break
                    elif(user_approval2 == "q"):
                        asdf # crash
                    else:
                        pass

                if(not(found_mask)):
                    success = False

            # Clear output for this candidate combination
            clear_output(wait=False)
                


        if(success):
            # All necessary images found; add target, then append id dictionary to download list
            to_download[scene_name] = scene_ids
            locations[scene_name] = location_coords
            found_any = True
            num_success += 1
            break


if(found_any):
    print("\nDone. Files to download: ")
    for k,v in to_download.items():
        print({k:v})

    if(save_meta):
        print("\nSaving metadata to image_ids.pkl.")
        with open('image_ids.pkl', 'wb') as fp:
            pickle.dump(to_download, fp)

else:
    print("Did not find any suitable images. ")

Reached maximum number of suitable scenes.

Done. Files to download: 
{'asia_cropland_china': {'target': 'S2B_MSIL2A_20211228T031129_N0301_R075_T50SKB_20211228T054139', '1w': 'S2B_MSIL2A_20211218T031129_N0301_R075_T50SKA_20211218T054114', '1m': 'S2B_MSIL2A_20211205T030109_N0301_R032_T50SKA_20211205T061129', '3m': 'S2A_MSIL2A_20211004T030551_N0301_R075_T50SKA_20211004T051723', '6m': 'S2A_MSIL2A_20210606T030541_N0300_R075_T50SKA_20210606T051217', '12m': 'S2A_MSIL2A_20210203T025931_N0214_R032_T50SKA_20210203T055016', 'mask': 'S2B_MSIL2A_20220226T030659_N0400_R075_T50SKB_20220226T055944'}}
{'america_cropland_iowa': {'target': 'S2A_MSIL2A_20220623T170901_N0400_R112_T15TVJ_20220623T231619', '1w': 'S2A_MSIL2A_20220620T165901_N0400_R069_T15TVJ_20220621T002016', '1m': 'S2A_MSIL2A_20220603T170901_N0400_R112_T15TVJ_20220603T234510', '3m': 'S2A_MSIL2A_20220315T171051_N0400_R112_T15TVJ_20220315T223410', '6m': 'S2B_MSIL2A_20211227T170719_N0301_R069_T15TVJ_20211227T195112', '12m': 'S2B_MSIL2A_2021062

**Sort scenes alphabetically in dict**

In [7]:
with open("image_ids_unsorted.pkl", 'rb') as fp:
    to_download = pickle.load(fp)

to_download2 = {k:v for k,v in sorted(to_download.items())}
for k,v in to_download2.items():
    print({k:v})

{'africa_forest_angola': {'target': 'S2B_MSIL2A_20220906T084559_N0400_R107_T33LYE_20220906T114544', 'l1c': 'S2B_MSIL1C_20220906T084559_N0400_R107_T33LYE_20220906T105206', 'sar': '', '1w': 'S2A_MSIL2A_20220901T084611_N0400_R107_T33LYE_20220901T155705', '1m': 'S2B_MSIL2A_20220817T084559_N0400_R107_T33LYE_20220817T120435', '3m': 'S2B_MSIL2A_20220618T084559_N0400_R107_T33LYE_20220618T121340', '6m': 'S2A_MSIL2A_20220315T084731_N0400_R107_T33LYE_20220315T115403', '12m': 'S2A_MSIL2A_20211016T084941_N0301_R107_T33LYE_20211016T114018', 'mask': 'S2B_MSIL2A_20230305T084749_N0509_R107_T33LYE_20230305T123021'}}
{'africa_forest_zambia': {'target': 'S2B_MSIL2A_20220907T081609_N0400_R121_T34LHK_20220907T111917', 'l1c': 'S2B_MSIL1C_20220907T081609_N0400_R121_T34LHK_20220907T102707', 'sar': '', '1w': 'S2B_MSIL2A_20220828T081609_N0400_R121_T34LHK_20220828T111354', '1m': 'S2B_MSIL2A_20220818T081609_N0400_R121_T35LKE_20220818T111217', '3m': 'S2A_MSIL2A_20220624T081621_N0400_R121_T34LHL_20220624T133918', '6

In [8]:
with open('image_ids.pkl', 'wb') as fp:
    pickle.dump(to_download2, fp)

In [13]:
with open('image_ids_unsorted.pkl', 'wb') as fp:
    pickle.dump(to_download, fp)

**Unused SH-based DL code (use SentinelSat instead for full products)**

In [11]:
# Download products SH

process_requests = []
meta = []

for location, scenes in to_download.items():
    print("Downloading images for scene: " + str(location))
    path = save_path + location + "/"
    if(not(os.path.exists(path))):
        os.mkdir(path)

    for timediff, img_id in scenes.items():
        path_local = path + "img_" + timediff + ".npy"
        if(not(os.path.exists(path_local)) or overwrite):
            timestamp_str = img_id[11:19]
            timestamp = datetime.datetime.strptime(timestamp_str, "%Y%m%d")
            interval = (timestamp - datetime.timedelta(days=2), timestamp + datetime.timedelta(days=2))

            bbox = BBox(bbox=locations[location], crs=CRS.WGS84)
            size = bbox_to_dimensions(bbox, resolution=20)

            if(timediff == "mask"):
                evscr = evalscript_s1
                dc = DataCollection.SENTINEL2_L1C
            else:
                evscr = evalscript
                dc = DataCollection.SENTINEL2_L2A

            request = SentinelHubRequest(
                evalscript=evscr,
                input_data=[
                    SentinelHubRequest.input_data(
                        data_collection=dc,
                        time_interval=interval,
                        #mosaicking_order=MosaickingOrder.LEAST_CC, 
                    )
                ],
                responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
                bbox=bbox,
                size=size,
                config=config,
            )
            process_requests.append(request)
            meta.append({'scene':location, 'img_type':timediff})

            img = request.get_data()[0]

            # Save as numpy file

            np.save(path_local, img)
